# Demonstration von 5 Ruff-Regelgruppen

In diesem Notebook werden fünf verschiedene Regelgruppen des Linters **Ruff** demonstriert:

1. **Security – S101**: Unsichere Nutzung von `assert`  
2. **Security – S506**: Unsichere Verwendung von `yaml.load()`  
3. **Formatierung – E711**: Falscher Vergleich mit `None` (`==` statt `is`)  
4. **Dokumentation – D100**: Fehlender Modul-Docstring  
5. **Performance – PERF203**: Verwendung von `try/except` innerhalb einer Schleife  

Für jede Regelgruppe werden folgende Schritte durchgeführt:  
- Erzeugung eines Beispiels mit **fehlerhaftem Code**  
- Erstellung einer **korrigierten Variante**  
- **Prüfung mit Ruff** und Darstellung der Ergebnisse  

Die Ausführung kann sowohl in **Google Colab** als auch in einem lokalen **Jupyter Notebook** erfolgen.


In [ ]:
# Installation von ruff (nur ausführen, wenn nicht installiert)
!pip install ruff --quiet

## Regelgruppe Security – 1. Beispiel S101 `assert`

In [16]:
# Fehlerhafte Version (unsicher)
with open("s101_assert_bad.py", "w", encoding="utf-8") as f:
    f.write("""# Unsichere Nutzung von assert (kann in optimiertem Modus ignoriert werden)
user_input = ""
assert user_input != "", "Input darf nicht leer sein!"  # unsicher
""")

# Ruff-Check (Security-Regeln)
!ruff check --select S s101_assert_bad.py

print("--------------------------------------------")

# Korrigierte Version (sicher)
with open("s101_assert_good.py", "w", encoding="utf-8") as f:
    f.write("""# Sichere Nutzung mit if + Exception
user_input = ""
if user_input == "":
    raise ValueError("Input darf nicht leer sein!")  # sicher
""")

# Ruff-Check (Security-Regeln)
!ruff check --select S s101_assert_good.py


s101_assert_bad.py:3:1: S101 Use of `assert` detected
  |
1 | # Unsichere Nutzung von assert (kann in optimiertem Modus ignoriert werden)
2 | user_input = ""
3 | assert user_input != "", "Input darf nicht leer sein!"  # unsicher
  | ^^^^^^ S101
  |

Found 1 error.
--------------------------------------------
All checks passed!


## Regelgruppe Security – 2. Beispiel S506  `yaml.load(...)`

In [14]:
# Fehlerhafte Version (unsicher)
with open("s506_yaml_bad.py", "w", encoding="utf-8") as f:
    f.write("""# Unsichere Nutzung von yaml.load (führt zu Sicherheitsrisiken)
import yaml

data = "!!python/object/apply:os.system ['echo unsicher']"
result = yaml.load(data)  # unsicher
print(result)
""")

# Ruff-Check (Security-Regeln)
!ruff check --select S s506_yaml_bad.py

print("--------------------------------------------")

# Korrigierte Version (sicher)
with open("s506_yaml_good.py", "w", encoding="utf-8") as f:
    f.write("""# Sichere Nutzung von yaml.safe_load
import yaml

data = "a: 1\\nb: 2"
result = yaml.safe_load(data)
print(result)
""")

# Ruff-Check (Security-Regeln)
!ruff check --select S s506_yaml_good.py


s506_yaml_bad.py:5:10: S506 Probable use of unsafe `yaml.load`. Allows instantiation of arbitrary objects. Consider `yaml.safe_load`.
  |
4 | data = "!!python/object/apply:os.system ['echo unsicher']"
5 | result = yaml.load(data)  # unsicher
  |          ^^^^^^^^^ S506
6 | print(result)
  |

Found 1 error.
--------------------------------------------
All checks passed!


## Regelgruppe E - 3. Beispiel E711

In [1]:
# Fehlerhafte Version (falscher Vergleich mit None)
with open("e711_bad.py", "w", encoding="utf-8") as f:
    f.write("""# Falscher Vergleich mit None
x = None
if x == None:
    print("x ist None")
""")

# Ruff-Check (nur Formatierungs-/Style-Regeln)
!ruff check --select E e711_bad.py

print("--------------------------------------------")

# Korrigierte Version (richtiger Vergleich mit None)
with open("e711_good.py", "w", encoding="utf-8") as f:
    f.write("""# Richtiger Vergleich mit None
x = None
if x is None:
    print("x ist None")
""")

# Ruff-Check (nur Formatierungs-/Style-Regeln)
!ruff check --select E e711_good.py


e711_bad.py:3:9: E711 Comparison to `None` should be `cond is None`
  |
1 | # Falscher Vergleich mit None
2 | x = None
3 | if x == None:
  |         ^^^^ E711
4 |     print("x ist None")
  |
  = help: Replace with `cond is None`

Found 1 error.
No fixes available (1 hidden fix can be enabled with the `--unsafe-fixes` option).
--------------------------------------------
All checks passed!


## Regelgruppe Dokumentation - 4. Beispiel D100

In [3]:
# Fehlerhafte Version (kein Modul-Docstring)
with open("d100_bad.py", "w", encoding="utf-8") as f:
    f.write("""def greet(name):
    \"\"\"Gibt einen Begrüßungstext aus.\"\"\"
    print(f"Hallo {name}")
""")

# Ruff-Check nur für D100
!ruff check --select D100 d100_bad.py

print("--------------------------------------------")

# Korrigierte Version (mit Modul-Docstring)
with open("d100_good.py", "w", encoding="utf-8") as f:
    f.write("""\"\"\"Dieses Modul enthält eine Begrüßungsfunktion.\"\"\"

def greet(name):
    \"\"\"Gibt einen Begrüßungstext aus.\"\"\"
    print(f"Hallo {name}")
""")

# Ruff-Check nur für D100
!ruff check --select D100 d100_good.py


d100_bad.py:1:1: D100 Missing docstring in public module
Found 1 error.
--------------------------------------------
All checks passed!


## Regelgruppe Performance – 5. Beispiel PERF203 `try/except`

In [7]:
# Fehlerhafte Version (try/except in einer Schleife → ineffizient)
with open("perf203_bad.py", "w", encoding="utf-8") as f:
    f.write("""# Schlechte Performance: try/except in der Schleife
for i in range(5):
    try:
        print(int("x"))  # Fehler provozieren
    except ValueError:
        print("Fehler bei Umwandlung")
""")

# Ruff-Check für Performance-Regeln
!ruff check --select PERF perf203_bad.py

print("--------------------------------------------")

# Korrigierte Version (try/except um die Schleife → effizienter)
with open("perf203_good.py", "w", encoding="utf-8") as f:
    f.write("""# Bessere Performance: try/except außerhalb der Schleife
try:
    for i in range(5):
        print(int("x"))  # Fehler provozieren
except ValueError:
    print("Fehler bei Umwandlung")
""")

# Ruff-Check für Performance-Regeln
!ruff check --select PERF perf203_good.py


perf203_bad.py:5:5: PERF203 `try`-`except` within a loop incurs performance overhead
  |
3 |       try:
4 |           print(int("x"))  # Fehler provozieren
5 | /     except ValueError:
6 | |         print("Fehler bei Umwandlung")
  | |______________________________________^ PERF203
  |

Found 1 error.
--------------------------------------------
All checks passed!
